# Transformace příznaků
Úloha zaměřená na implementaci metod PCA a LDA pro transformaci příznaků. 

Výchozí motivace
- Cílem je provést dekorelaci příznaků a vybrat pouze ty nejvýznamnější

Chceme
- Snížit zátěž (výpočetní, časovou) vlastního klasifikačního procesu.
- Zvýšit úspěšnost klasifikace

## Data

In [1]:
import numpy as np
import usu

npzfile = np.load("data/data_12.npz")
npzfile.files


['testData', 'testRef', 'trainData', 'trainRef']

In [2]:
testData = npzfile["testData"]
testRef = npzfile["testRef"]

trainData = npzfile["trainData"]
trainRef = npzfile["trainRef"]

trainData.shape, trainRef.shape, testData.shape, testRef.shape

((1050, 5), (1050, 1), (850, 5), (850, 1))

### Výpočet úspěšnosti
$$ accuracy = \frac{\text{počet správně klasifikovaných objektů}}{\text{počet všech klasifikovaných  objektů}} $$

In [3]:
def accuracy(testRef, predRef):
    """
    vraci uspesnost v procentech
    """
    return (predRef == testRef).mean() * 100


### Vzdálenostní funkce
V dané uloze implemetujeme jenom euklidovskou vzdálenost

#### Euklidovská vzdálenost (L2)
$$ d(x,z) = \sqrt{\sum_{i=0}^{Dim}{(x_i - z_i)^2}} $$

Při implementaci jde vynechat operaci druhé odmocniny, protože druhá odmocnina je monotónní rostoucí funkce. To znamená, že se mění jenom absolutní hodnoty vzdálenosti, ale pořadí se zachovává:
$$ d(x,z) = \sum_{i=0}^{Dim}{(x_i - z_i)^2} $$

In [4]:
def euclidian_distance(testItem, trainData):
    """vypocet vzdalenosti jendoho testovaciho vzorku ke vsem trenovacim datum"""
    distances = np.sum((trainData - testItem) ** 2, axis=1)

    return distances


euclidian_distance(testData[0], trainData)

array([38.54881076, 54.30823974, 56.90800423, ..., 39.14515371,
       20.25885791, 17.23315459], shape=(1050,))

In [5]:
def getPrediction(trainData, trainRef, testData):
    # pomoci funkce euclidian_distance

    nTestElements = np.size(testData, 0)
    classes = np.unique(trainRef)
    nClasses = np.size(classes, 0)
    predRef = np.zeros([nTestElements, 1])

    # pomoci etalonu
    etalons = np.zeros((nClasses, trainData.shape[1]))

    for i in range(nClasses):
        indices = trainRef.ravel() == classes[i]
        data_for_class = trainData[indices]
        etalons[i] = np.mean(data_for_class, axis=0)

    for i in range(nTestElements):
        dists = euclidian_distance(testData[i], etalons)
        min_index = np.argmin(dists)
        predRef[i] = classes[min_index]

    return predRef

In [6]:
predRef = getPrediction(trainData, trainRef, testData)
print(f"acc : {accuracy(testRef, predRef):.2f}")

acc : 93.06


## Metody transformace příznaků
### Transformace na základě dekorelace (a následná redukce): PCA
- Z dat $X$ určíme kovariační matici $\Sigma_x$
- Vypočteme její vlastní vektory $e$ a sestavíme z nich matici $E$
- Pak transformovaná data:

$$X_{tr} = X (E_{0:f})^T$$
kde f je počet příznaků
- **Kovariační matice transformovaných dat bude diagonální**
- **Data v novém souřadném systému budou dekorelovaná**


### Transformace s ohledem na co největší diskriminativnost: LDA
Vypočteme matici vlastních vektorů $E$ z matice určené součinem $ \Sigma_{bc} \Sigma_{wc}^{-1} $

kde 
- kovarianční matice spočítaná ze středních hodnot tříd: $$\Sigma_{bc} = \frac{1}{N} \sum_{class=0}^{C} N_{class} (\mu_{class} - \mu)^T (\mu_{class} - \mu)$$ 
- průměrná kovarianční matice tříd: $$\Sigma_{wc} = \frac{1}{N} \sum_{class=0}^{C}{N_{class} \Sigma_{class}}$$ 


kde N je počet prvků a C je počet tříd

- Pak transformovaná data:

$$X_{tr} = X (E_{0:f})^T$$

In [7]:
class transform:
    def __init__(self, trainData, trainRef, testData, testRef):
        self.trainData = trainData
        self.testData = testData
        self.trainRef = trainRef
        self.testRef = testRef

        cov_matrix = np.cov(self.trainData, rowvar=False)
        eig_values, eig_vectors = np.linalg.eig(cov_matrix)

        sorted_indices = np.argsort(eig_values)[::-1]
        self.sorted_eig_vectors = eig_vectors[:, sorted_indices]

    def pca(self, data, nFeautures=1):
        """
        Transformuje data PCA transformacni matici.
        """
        # redukce
        eigenvector_subset = self.sorted_eig_vectors[:, :nFeautures]
        transformed_data = data @ eigenvector_subset

        return transformed_data

    def lda(self, data, nFeautures=1):
        """
        transformuje data lda transformacni matici
        """
        ...

        return ...


In [8]:
transformation = transform(trainData, trainRef, testData, testRef)

In [9]:
#pca
for dim in range(1,np.size(trainData,1)+1):
    trainDataT = transformation.pca(trainData, dim)
    testDataT = transformation.pca(testData, dim)
    
    predRef = getPrediction(trainDataT,trainRef,testDataT)
    print(f"priznaky: {dim} -> acc : {accuracy(testRef,predRef):.2f} %")

priznaky: 1 -> acc : 70.24 %
priznaky: 2 -> acc : 88.71 %
priznaky: 3 -> acc : 92.82 %
priznaky: 4 -> acc : 92.94 %
priznaky: 5 -> acc : 93.06 %


In [10]:
#lda
for dim in range(1,np.size(trainData,1)+1):
    trainDataT = transformation.lda(trainData, dim)
    testDataT = transformation.lda(testData, dim)
    
    predRef = getPrediction(trainDataT,trainRef,testDataT)
    print(f"priznaky: {dim} -> acc : {accuracy(testRef,predRef):.2f} %")
  

IndexError: tuple index out of range

In [ ]:
#Závěr:
#Kdy použít kterou metodu?
